<a href="https://colab.research.google.com/github/chetools/CHE4061_Fall2025/blob/main/VLE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -N -q https://raw.githubusercontent.com/chetools/chetools/main/tools/che5.ipynb -O che5.ipynb
!pip install importnb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.8 MB/s eta 0:00:00


In [2]:
from importnb import Notebook
with Notebook():
    from che5 import Props

import numpy as np
import scipy as sp
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default = "plotly_dark"
import jax
import jax.numpy as jnp
jax.config.update("jax_enable_x64", True)

In [7]:
p=Props(['Methanol','Water'])

In [8]:
p.Pvap(np.linspace(273.15,373.15,50))

Array([[  3947.04688179,    614.65936223],
       [  4494.62332748,    712.03065654],
       [  5106.86844911,    822.80507403],
       [  5790.01261566,    948.53690451],
       [  6550.73821014,   1090.92448975],
       [  7396.1997174 ,   1251.81977391],
       [  8334.04386573,   1433.23816234],
       [  9372.42977908,   1637.36867627],
       [ 10520.04909715,   1866.58438982],
       [ 11786.14602125,   2123.45313443],
       [ 13180.53724498,   2410.74845458],
       [ 14713.6317297 ,   2731.46079754],
       [ 16396.45028639,   3088.80891885],
       [ 18240.64492698,   3486.25148423],
       [ 20258.51794991,   3927.49884777],
       [ 22463.04072673,   4416.52498544],
       [ 24867.87215857,   4957.57956231],
       [ 27487.37677347,   5555.20011123],
       [ 30336.64243783,   6214.22430028],
       [ 33431.49765778,   6939.80226589],
       [ 36788.52844865,   7737.40898831],
       [ 40425.09475318,   8612.85668593],
       [ 44359.3463918 ,   9572.30720467],
       [ 48

In [21]:
z=np.array([0.4,0.6])

In [24]:
1/np.sum(z/p.Pvap(300))

Array(5250.07748118, dtype=float64)